# Biol 359A | Regularization
### Spring 2025, Week 4
Objectives:
- Understand the Bias-Variance Tradeoff: Learn how increasing model complexity impacts bias and variance.
- Gain intuition for Regularization Techniques: Implement Ridge and LASSO to prevent overfitting.
- Implement Model Evaluation and Selection: Employ train-test splits for optimal model selection.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display


### Generate synthetic data

Today we will start by working with in-silico data. The code below will generate the data.

In [ ]:
def generate_data(n_samples=100, n_features=10, noise_level=0.5, random_state=42, add_n_extreme=0):
    """
    Generate synthetic data with controlled noise and multicollinearity.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    n_features : int
        Number of features to generate
    noise_level : float
        Standard deviation of the Gaussian noise
    random_state : int
        Random seed for reproducibility
    add_extreme : bool
        If True, adds an extreme outlier value to the last sample
    
    Returns:
    --------
    X : ndarray of shape (n_samples, n_features)
        Feature matrix
    y : ndarray of shape (n_samples,)
        Target vector
    beta : ndarray of shape (n_features,)
        True coefficients
    """
    # Set random seed
    np.random.seed(random_state)
    
    # Generate random feature matrix
    X = np.random.randn(n_samples, n_features)
    
    # Add multicollinearity: Make some features correlated
    X[:, 5] = 0.7 * X[:, 0] + 0.3 * np.random.randn(n_samples)
    X[:, 6] = 0.8 * X[:, 1] + 0.2 * np.random.randn(n_samples)
    X[:, 7] = 0.9 * X[:, 2] + 0.1 * np.random.randn(n_samples)
    
    # Generate true coefficients with sparsity (some are zero)
    beta = np.zeros(n_features)
    beta[0] = 1.5
    beta[1] = -2.0
    beta[2] = 3.0
    beta[3] = -0.5
    beta[4] = 1.0
    # Features 5, 6, 7 have zero coefficients (but are correlated with others)
    beta[8] = -1.0
    beta[9] = 2.0
    
    # Generate target variable with noise
    y = np.dot(X, beta) + noise_level * np.random.randn(n_samples)
    # Add an extreme outlier if requested
    if add_n_extreme:
        #y[-add_n_extreme:] += noise_level * (np.random.rand(add_n_extreme)) * 50
        X[-add_n_extreme:] += np.random.randn(add_n_extreme, n_features) * 10
    
    return X, y, beta

In [ ]:
# Generate the data
n_samples = 100
n_features = 10
noise_level = 0.5
X, y, true_beta = generate_data(n_samples, n_features, noise_level, add_n_extreme=10)
true_beta

### Multi linear regression

We want to build a linear regression model that will help us learn about the system that generated these data. To make the coefficients of our model more comparable to each other, we need to start by standardizing our data.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features for better convergence in regularization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Before standarization:")
print(f"mean: {np.mean(X_train): .3f}, std: {np.std(X_train):.3f}")
print("After standarization:")
print(f"mean: {np.mean(X_train_scaled): .3f}, std: {np.std(X_train_scaled):.3f}")

In [ ]:
def fit_mlr(X_train, y_train, X_test, y_test):
    """Fit Multiple Linear Regression model and return metrics"""
    mlr = LinearRegression()
    mlr.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = mlr.predict(X_train)
    y_test_pred = mlr.predict(X_test)
    
    # Performance metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    return mlr.coef_, train_mse, test_mse, train_r2, test_r2

def plot_mlr_results(noise_level=0.5, n_samples=100, add_n_extreme=0):
    """Plot MLR results with the given noise level and sample size"""
    # Generate new data with the specified parameters
    X, y, true_beta = generate_data(n_samples=n_samples, noise_level=noise_level, add_n_extreme=add_n_extreme)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit MLR
    coef, train_mse, test_mse, train_r2, test_r2 = fit_mlr(X_train_scaled, y_train, X_test_scaled, y_test)
    
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Coefficient values comparison (True vs Estimated)
    feature_indices = np.arange(len(true_beta))
    bar_width = 0.35
    
    ax1.bar(feature_indices - bar_width/2, true_beta, bar_width, label='True Coefficients', color='blue', alpha=0.7)
    ax1.bar(feature_indices + bar_width/2, coef, bar_width, label='Estimated Coefficients', color='red', alpha=0.7)
    
    ax1.set_xlabel('Feature Index')
    ax1.set_ylabel('Coefficient Value')
    ax1.set_title('True vs. Estimated Coefficients in MLR')
    ax1.set_xticks(feature_indices)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Predicted vs Actual values
    ax2.scatter(y_train, np.dot(X_train_scaled, coef), color='k', alpha=0.7, label="Train")
    ax2.scatter(y_test, np.dot(X_test_scaled, coef), color='green', alpha=0.7, label="Test")
    
    # Add a diagonal line (perfect predictions)
    min_val_train = min(min(y_train), min(np.dot(X_train_scaled, coef)))
    min_val_test = min(min(y_test), min(np.dot(X_test_scaled, coef)))
    min_val = min(min_val_train, min_val_test)
    max_val_train = max(max(y_train), max(np.dot(X_train_scaled, coef)))
    max_val_test = max(max(y_test), max(np.dot(X_test_scaled, coef)))
    max_val = max(max_val_test, max_val_test)
    ax2.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
    
    ax2.set_xlabel('Actual Values')
    ax2.set_ylabel('Predicted Values')
    ax2.set_title('Predicted vs. Actual Values in MLR')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    # Add text with performance metrics
    textstr = '\n'.join((
        f'Training MSE: {train_mse:.2f}',
        f'Test MSE: {test_mse:.2f}',
        f'Training R²: {train_r2:.2f}',
        f'Test R²: {test_r2:.2f}'))
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    
    plt.tight_layout()
    plt.show()

# Interactive MLR visualization
print("Multiple Linear Regression Interactive Visualization")
print("===================================================")
print("Adjust the noise level and sample size to see how they affect MLR performance.")

interact(plot_mlr_results,
         add=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Noise Level:'),
         noise_level=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Noise Level:'),
         n_samples=widgets.IntSlider(min=50, max=500, step=50, value=100, description='Sample Size:'),
         add_n_extreme=widgets.IntSlider(min=0, max=10, step=1, value=0, description='N Outliers:'));

### Ridge regression

The interactive visualization below performs Ridge regression on these data. Use the sliders to adjust the polynomial degree and the lambda value, which controls the strength of the regularization:

- Polynomial Degree: Influences model complexity. Lower degrees might not capture all the data variability (underfitting), while higher degrees might model the noise as well (overfitting).
- Lambda Value: Adjusts the Ridge regularization strength. Increasing lambda enhances the model's generalization by penalizing large coefficients, helping to prevent overfitting.

Observe how adjusting the lambda values impacts the coefficients.

In [ ]:
def fit_ridge(X_train, y_train, X_test, y_test, alpha):
    """Fit Ridge Regression model and return metrics"""
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = ridge.predict(X_train)
    y_test_pred = ridge.predict(X_test)
    
    # Performance metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    return ridge.coef_, train_mse, test_mse, train_r2, test_r2

def plot_ridge_results(alpha=1.0, noise_level=0.5, n_samples=100, add_n_extreme=0):
    """Plot Ridge Regression results with the given parameters"""
    # Generate new data with the specified parameters
    X, y, true_beta = generate_data(n_samples=n_samples, noise_level=noise_level, add_n_extreme=add_n_extreme)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit MLR for comparison
    mlr_coef, mlr_train_mse, mlr_test_mse, mlr_train_r2, mlr_test_r2 = fit_mlr(X_train_scaled, y_train, X_test_scaled, y_test)
    mlr_y_test_pred = np.dot(X_test_scaled, mlr_coef)
    # Fit Ridge
    ridge_coef, ridge_train_mse, ridge_test_mse, ridge_train_r2, ridge_test_r2 = fit_ridge(X_train_scaled, y_train, X_test_scaled, y_test, alpha)
    ridge_y_test_pred = np.dot(X_test_scaled, ridge_coef)


    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Coefficient values comparison
    feature_indices = np.arange(len(true_beta))
    bar_width = 0.25
    
    ax1.bar(feature_indices - bar_width, true_beta, bar_width, label='True Coefficients', color='blue', alpha=0.7)
    ax1.bar(feature_indices, mlr_coef, bar_width, label='MLR Coefficients', color='red', alpha=0.7)
    ax1.bar(feature_indices + bar_width, ridge_coef, bar_width, label='Ridge Coefficients', color='green', alpha=0.7)
    
    ax1.set_xlabel('Feature Index')
    ax1.set_ylabel('Coefficient Value')
    ax1.set_title(f'Coefficient Comparison (Ridge with α={alpha:.2f})')
    ax1.set_xticks(feature_indices)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Regression results - Predicted vs Actual
    ax2.scatter(y_test, mlr_y_test_pred, color='red', alpha=0.7, label='MLR Predictions')
    ax2.scatter(y_test, ridge_y_test_pred, color='green', alpha=0.7, label='Ridge Predictions')
    
    # Add a diagonal line (perfect predictions)
    min_val = min(min(y_test), min(min(mlr_y_test_pred), min(ridge_y_test_pred)))
    max_val = max(max(y_test), max(max(mlr_y_test_pred), max(ridge_y_test_pred)))
    ax2.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2, label='Perfect Predictions')
    
    ax2.set_xlabel('Actual Values')
    ax2.set_ylabel('Predicted Values')
    ax2.set_title('Predicted vs. Actual Values: MLR vs Ridge')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Add text with performance metrics
    mlr_metrics = f'MLR: MSE={mlr_test_mse:.2f}, R²={mlr_test_r2:.2f}'
    ridge_metrics = f'Ridge: MSE={ridge_test_mse:.2f}, R²={ridge_test_r2:.2f}'
    
    ax2.text(0.05, 0.95, mlr_metrics, transform=ax2.transAxes, fontsize=10,
            verticalalignment='top', color='red', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    ax2.text(0.05, 0.87, ridge_metrics, transform=ax2.transAxes, fontsize=10,
            verticalalignment='top', color='green',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    plt.tight_layout()
    plt.show()
    
    # Display regularization effect summary
    print(f"Ridge Regression (α={alpha:.2f}) Summary:")
    print(f"L2 Penalty Term: {np.sum(ridge_coef**2):.4f}")
    print(f"Test MSE Improvement: {mlr_test_mse - ridge_test_mse:.4f}")



# Interactive Ridge Regression visualization
print("Ridge Regression Interactive Visualization")
print("==========================================")
print("Adjust the alpha parameter to control regularization strength,")
print("and other parameters to see their effect on Ridge Regression performance.")

interact(plot_ridge_results, 
         alpha=widgets.FloatLogSlider(min=-2, max=3, value=1.0, base=10, description='Alpha (λ):'),
         noise_level=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Noise Level:'),
         n_samples=widgets.IntSlider(min=50, max=500, step=50, value=100, description='Sample Size:'),
         add_n_extreme=widgets.IntSlider(min=0, max=10, step=1, value=0, description='N Outliers:'));

## The Bias-Variance Tradeoff

- Bias: Bias is the error introduced by approximating a real-world problem, which might be highly complex, with a simpler model. It reflects how well the model captures the underlying patterns of the data. High bias can cause the model to miss relevant relations between features and target outputs (underfitting).

- Variance: Variance measures how much the model's predictions vary for a given dataset. A model with high variance pays a lot of attention to training data and learns noise as well as signals, leading to less generalizable models on new data (overfitting).

The interactive plot below allows you to explore this concept. The interactive slider allows you to adjust the degree of the polynomial regression model applied to synthetic data. The polynomial features (like x, x^2) and the corresponding model coefficients are displayed in the model equation and the resulting model predictions are plotted on the synthetic data. As you increase the polynomial degree, observe how the model predictions change:

- Low-degree polynomials may show high bias, where the model is too simplistic to capture the essential patterns in the data, resulting in a poor fit.
- High-degree polynomials may exhibit high variance, fitting not only the underlying data pattern but also the noise. This is seen as the model begins to closely trace minor fluctuations in the data, indicating overfitting.

As you adjust the slider, observe changes in the Mean Squared Error (MSE) and R-squared values below the plot. The MSE will help you understand the average squared difference between the observed and predicted values, while the R-squared value indicates the proportion of the variance in the dependent variable that is predictable from the independent variable.

In [ ]:
def plot_polynomial_complexity_MLR(degree):
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly = polynomial_features.fit_transform(x_standardized[:, np.newaxis])
    
    model = LinearRegression()
    model.fit(x_poly, y_standardized)
    y_poly_pred = model.predict(x_poly)
    
    # Generating polynomial feature names and equation
    feature_names = ['x^{}'.format(i) if i > 1 else 'x' for i in range(1, degree + 1)]
    coefficients = model.coef_
    equation_terms = [f"{coeff:.3f}*{name}" for coeff, name in zip(coefficients, feature_names)]
    equation = " + ".join(equation_terms)
    wrapped_equation = '\n               '.join(textwrap.wrap(equation, width=80))  # Wrap the equation
    
    # Print the complete equation
    print("Equation: y = " + wrapped_equation)
    
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.scatter(x_standardized, y_standardized, s=20, label="Noisy Data Points")
    plt.plot(x_standardized, y_poly_pred, color='r', label=f"Polynomial Degree {degree} Fit")
    plt.xlabel("Feature X")
    plt.ylabel("Target Y")
    plt.title("Polynomial Regression Fit")
    plt.legend()
    plt.show()
    
    # Calculate and print errors as a measure of Bias and Variance
    mse = mean_squared_error(y_standardized, y_poly_pred)
    rsquared = model.score(x_poly, y_standardized)
    print(f"Mean Squared Error (MSE) for Degree {degree}: {mse:.2f}")
    print(f"R-Squared for Degree {degree}: {rsquared:.2f}")

# Create interactive widget
degree_slider = IntSlider(min=1, max=11, step=1, value=1, description="Polynomial Degree",
                          style={'description_width': 'initial'},
                          layout={'width': '500px'})
interactive_plot = interactive(plot_polynomial_complexity_MLR, degree=degree_slider)
interactive_plot

DISCUSSION QUESTION:
- Are there $\beta_0$ terms in any of the model equations? Why or why not?

ASSIGNMENT QUESTIONS:
- Please answer questions 11, 12, and 13 in the assignment

#### Ridge regression

The interactive visualization below performs Ridge regression on these data. Use the sliders to adjust the polynomial degree and the lambda value, which controls the strength of the regularization:

- Polynomial Degree: Influences model complexity. Lower degrees might not capture all the data variability (underfitting), while higher degrees might model the noise as well (overfitting).
- Lambda Value: Adjusts the Ridge regularization strength. Increasing lambda enhances the model's generalization by penalizing large coefficients, helping to prevent overfitting.

Observe how adjusting the lambda values impacts the coefficients.

In [ ]:
def plot_polynomial_complexity_Ridge(degree, alpha):
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly = polynomial_features.fit_transform(x_standardized[:, np.newaxis])
    
    # Using Ridge regression model
    model = Ridge(alpha=alpha)
    model.fit(x_poly, y_standardized)
    y_poly_pred = model.predict(x_poly)
    
    # Generating polynomial feature names and equation
    feature_names = ['x^{}'.format(i) if i > 1 else 'x' for i in range(1, degree + 1)]
    coefficients = model.coef_
    equation_terms = [f"{coeff:.3f}*{name}" for coeff, name in zip(coefficients, feature_names)]
    equation = " + ".join(equation_terms)
    wrapped_equation = '\n               '.join(textwrap.wrap(equation, width=80))  # Wrap the equation
    
    # Print the complete equation
    print("Equation: y = " + wrapped_equation)
    
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.scatter(x_standardized, y_standardized, s=20, label="Standardized Data Points")
    plt.plot(x_standardized, y_poly_pred, color='r', label=f"Ridge Degree {degree} Fit, Alpha {alpha}")
    plt.xlabel("Standardized Feature X")
    plt.ylabel("Standardized Target Y")
    plt.title("Ridge Polynomial Regression Fit")
    plt.legend()
    plt.show()
    
    # Calculate and print errors as a measure of Bias and Variance
    mse = mean_squared_error(y_standardized, y_poly_pred)
    rsquared = model.score(x_poly, y_standardized)
    print(f"Mean Squared Error (MSE) for Degree {degree}: {mse:.2f}")
    print(f"R-Squared for Degree {degree}: {rsquared:.2f}")

# Create interactive widgets
degree_slider = IntSlider(min=1, max=11, step=1, value=1, description="Polynomial Degree",
                          style={'description_width': 'initial'},
                          layout={'width': '500px'})
lambda_slider = FloatSlider(min=.01, max=1, step=.01, value=.1, description="Lambda",
                         style={'description_width': 'initial'},
                         layout={'width': '500px'})

# Keen observers may notice that we rename lambda to alpha in the function (see below). This is because lambda is a reserved keyword in Python.
interactive_plot = interactive(plot_polynomial_complexity_Ridge, degree=degree_slider, alpha=lambda_slider)
interactive_plot


ASSIGNMENT QUESTION:
- Please answer question 14 in the assignment

#### Lasso

This interactive visualization showcases Lasso regression applied to polynomial models. Adjust the polynomial degree and alpha value using the provided sliders to explore their effects:

- Polynomial Degree: Controls the complexity of the model. Lower degrees might result in underfitting, where the model is too simplistic, while higher degrees might lead to overfitting, capturing noise instead of just the underlying data pattern.
- Lambda Value: Manages the Lasso regularization strength, promoting sparsity in the model coefficients. Higher lambda values can lead to more coefficients being reduced to zero, simplifying the model and potentially improving its generalizability.

Observe how changing the lambda value affects the number of parameters in the model.

In [ ]:
def plot_polynomial_complexity_Lasso(degree, alpha):
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly = polynomial_features.fit_transform(x_standardized[:, np.newaxis])
    
    # Using Lasso regression model
    model = Lasso(alpha=alpha, max_iter=10000)
    model.fit(x_poly, y_standardized)
    y_poly_pred = model.predict(x_poly)
    
    # Generating polynomial feature names and equation
    feature_names = ['x^{}'.format(i) if i > 1 else 'x' for i in range(1, degree + 1)]
    coefficients = model.coef_
    equation_terms = [f"{coeff:.3f}*{name}" if coeff != 0 else "" for coeff, name in zip(coefficients, feature_names)]
    equation_terms = [term for term in equation_terms if term]  # Remove empty strings
    equation = " + ".join(equation_terms)
    wrapped_equation = '\n               '.join(textwrap.wrap(equation, width=80))  # Wrap the equation
    
    # Print the complete equation
    print("Equation: y = " + wrapped_equation)
    
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.scatter(x_standardized, y_standardized, s=20, label="Standardized Data Points")
    plt.plot(x_standardized, y_poly_pred, color='r', label=f"Lasso Degree {degree} Fit, Alpha {alpha}")
    plt.xlabel("Standardized Feature X")
    plt.ylabel("Standardized Target Y")
    plt.title("Lasso Polynomial Regression Fit")
    plt.legend()
    plt.show()
    
    # Calculate and print errors as a measure of Bias and Variance
    mse = mean_squared_error(y_standardized, y_poly_pred)
    rsquared = model.score(x_poly, y_standardized)
    print(f"Mean Squared Error (MSE) for Degree {degree}: {mse:.2f}")
    print(f"R-Squared for Degree {degree}: {rsquared:.2f}")

# Create interactive widgets
degree_slider = IntSlider(min=1, max=11, step=1, value=1, description="Polynomial Degree",
                          style={'description_width': 'initial'},
                          layout={'width': '500px'})
lambda_slider = FloatSlider(min=0.01, max=1, step=.01, value=.1, description="Lambda",
                         style={'description_width': 'initial'},
                         layout={'width': '500px'})

interactive_plot = interactive(plot_polynomial_complexity_Lasso, degree=degree_slider, alpha=lambda_slider)
interactive_plot

ASSIGNMENT QUESTION
- Please answer question 15 in the assignment

Now we are not going to visualize the model fit with the data, we are just going to return a table. This interactive tool allows you to compare three different regression models: Standard Multilinear Regression (MLR), Lasso, and Ridge. Each model is evaluated using polynomial features of the data, which you can adjust using the provided sliders.

Use the sliders to:
- Adjust the Polynomial Degree: Use the degree slider to change the complexity of the polynomial features from 1 to 11. This adjustment affects how well each model can potentially fit the data.
- Set the Regularization Strength (Lambda): The lambda slider controls the alpha value for the Lasso and Ridge models. It adjusts the intensity of the penalty applied to the models' coefficients to prevent overfitting. For Standard MLR, this setting has no effect as there is no regularization.

The results table reports:
- Sum of Squared Errors (SSE): the total error of the predictions. Lower values indicate a better fit to the data.
- Penalties (L1 and L2): the sum of the absolute values of the coefficients (L1) and the sum of the squares of the coefficients (L2). These values are relevant for Lasso and Ridge, respectively, and help in assessing the impact of regularization.
- Number of Parameters: how many coefficients are actively used in the model. Lasso may set some coefficients to zero, reducing the effective number of parameters.
- Full Cost: combines the SSE with the penalties, giving a comprehensive measure of model performance considering both fit and complexity.

Review the full cost and the number of parameters to gauge each model's efficiency and complexity. Consider models with lower SSE and appropriate costs, which balance fit and simplicity, to prevent overfitting. Use the R-squared values to understand how well each model's variability explains the dependent variable.

Choose the model (type, polynomial degree, and lambda if relevant) that you believe best fits the data based on these metrics. Plot that model's parity plot using the next code cell.

In [ ]:
def compare_models(degree, alpha_value):
    # Prepare the polynomial features
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly = polynomial_features.fit_transform(x_standardized[:, np.newaxis])

    # Initialize models with a dynamic alpha value for regularization
    models = {
        'Linear': LinearRegression(),
        'Lasso': Lasso(alpha=alpha_value, max_iter=10000),
        'Ridge': Ridge(alpha=alpha_value)
    }

    # Table to store results
    results = []

    # Fit models and calculate errors and penalties
    for name, model in models.items():
        model.fit(x_poly, y_standardized)
        y_poly_pred = model.predict(x_poly)

        # Calculating SSE
        sse = np.sum((y_standardized - y_poly_pred) ** 2)

        # Calculating L1 and L2 penalties
        L1_penalty = np.sum(np.abs(model.coef_))
        L2_penalty = np.sum(np.square(model.coef_))

        # Number of parameters
        num_params = len([param for param in model.coef_ if param != 0])
        
        results.append({
            'Model': name,
            'SSE': sse,
            'Penalty': 'L1' if name == 'Lasso' else 'L2' if name == 'Ridge' else 'None',
            'L1': L1_penalty,
            'L2': L2_penalty,
            'Number of Parameters' : num_params,
        })

    # Create a DataFrame to display the results
    results_df = pd.DataFrame(results)
    display(results_df.set_index('Model'))  # Use display to show the DataFrame in the interactive output

# Create interactive widgets for the degree of the polynomial and alpha value
degree_slider = IntSlider(min=1, max=11, step=1, value=3, description="Polynomial Degree",
                          style={'description_width': 'initial'},
                          layout={'width': '500px'})
lambda_slider = FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description="Lambda",
                           style={'description_width': 'initial'},
                           layout={'width': '500px'})

interactive_plot = interactive(compare_models, degree=degree_slider, alpha_value=lambda_slider)
interactive_plot

In [ ]:
def plot_parity(model_type, degree, alpha_value):
    # Prepare the polynomial features
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly = polynomial_features.fit_transform(x_standardized[:, np.newaxis])

    # Choose the model based on user input
    if model_type == 'Standard MLR':
        model = LinearRegression()
    elif model_type == 'Lasso':
        model = Lasso(alpha=alpha_value, max_iter=10000)
    elif model_type == 'Ridge':
        model = Ridge(alpha=alpha_value)

    # Fit the model
    model.fit(x_poly, y_standardized)
    y_poly_pred = model.predict(x_poly)

    # Calculating the R^2 score
    r_squared = r2_score(y_standardized, y_poly_pred)

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(y_standardized, y_poly_pred, alpha=0.7, color='dodgerblue', label=f'Predicted vs. Actual')
    plt.plot(y_standardized, y_standardized, color='red', linestyle='--', label='Ideal Fit')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Parity Plot: {model_type} (Degree {degree}, Lambda {alpha_value})\n$R^2={r_squared:.3f}$')
    plt.legend()
    plt.grid(True)
    plt.show()

# Creating interactive widgets
model_dropdown = Dropdown(
    options=['Standard MLR', 'Lasso', 'Ridge'],
    value='Standard MLR',
    description='Model Type:',
    disabled=False,
)

degree_slider = IntSlider(
    min=1, 
    max=12, 
    step=1, 
    value=3, 
    description="Degree",
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

lambda_slider = FloatSlider(
    min=0.01, 
    max=1.0, 
    step=0.01, 
    value=0.1, 
    description="Lambda",
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)

# Create interactive output
interactive_output = interactive(plot_parity, model_type=model_dropdown, degree=degree_slider, alpha_value=lambda_slider)

# Display the interactive output and sliders
display(interactive_output)

Discussion question:
- What degree did you choose? Why?

Great, Now we are going to pretend we gathered a lot more data of the same system. Set the variables in the first three lines to make the function plot the model you selected in the exercise above. How good is the fit? What changes can you make to the model to improve the predictions?

In [ ]:
# SET HYPERPARAMETERS HERE:
initial_model_type = 'Standard MLR' # Options are 'Standard MLR', 'Lasso', 'Ridge'
initial_degree = 1
initial_lambda = 0.1
# ------------------------------------

# Generate more data
x_new = np.linspace(0, 10, 100)  # More data points for better visualization
y_new = 0.0005*x_new**5 - 0.001*x_new**4 + 0.002*x_new**3 - 0.001*x_new**2 + 0.005*x_new + 2
noise_new = np.random.randn(100) * 3  # Same noise level
y_noisy_new = y_new + noise_new

# Standardize the new data
scaler_x = StandardScaler()
x_standardized_new = scaler_x.fit_transform(x_new[:, np.newaxis]).flatten()
scaler_y = StandardScaler()
y_standardized_new = scaler_y.fit_transform(y_noisy_new[:, np.newaxis]).flatten()

def plot_model_predictions(model_type, degree, alpha):
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly_new = polynomial_features.fit_transform(x_standardized_new[:, np.newaxis])
    
    if model_type == 'Standard MLR':
        model = LinearRegression()
    elif model_type == 'Lasso':
        model = Lasso(alpha=alpha, max_iter=10000)
    elif model_type == 'Ridge':
        model = Ridge(alpha=alpha)
        
    model.fit(x_poly_new, y_standardized_new)
    y_poly_pred_new = model.predict(x_poly_new)

    plt.figure(figsize=(10, 6))
    plt.scatter(x_standardized_new, y_standardized_new, color='blue', label='Actual Data')
    plt.plot(x_standardized_new, y_poly_pred_new, color='red', label='Model Prediction')
    plt.title(f'Model Type: {model_type}, Degree: {degree}, Lambda: {alpha}\nR^2: {r2_score(y_standardized_new, y_poly_pred_new):.3f}')
    plt.xlabel('Standardized X')
    plt.ylabel('Standardized Y')
    plt.legend()
    plt.show()

# Interactive widgets setup
model_dropdown = Dropdown(options=['Standard MLR', 'Lasso', 'Ridge'], value=initial_model_type, description='Model Type:')
degree_slider = IntSlider(min=1, max=11, value=initial_degree, description='Polynomial Degree', continuous_update=False)
lambda_slider = FloatSlider(min=0.01, max=1.0, value=initial_lambda, step=0.01, description='Lambda', continuous_update=False)

interactive_plot = interactive(plot_model_predictions, model_type=model_dropdown, degree=degree_slider, alpha=lambda_slider)
display(interactive_plot)


Most of the time, we can't collect a bunch of new data to see how well our model generalizes. Instead, we have to split our existing data into "training data", which we use to build our model and select which model we think best describes our data, and "testing data" which we use to assess our model's generalizability. The code below splits the data from above into a training data set (which is a randomly selected 70% of the data) and a testing set (the remaining 30%)

In [ ]:
# Separates data into training and test sets
def train_test_split(x, y, test_size=0.2):
    # Shuffle the data
    indices = np.random.permutation(len(x))
    x_shuffled = x[indices]
    y_shuffled = y[indices]
    
    # Split the data into training and test sets
    split_index = int(len(x) * (1 - test_size))
    x_train, x_test = x_shuffled[:split_index], x_shuffled[split_index:]
    y_train, y_test = y_shuffled[:split_index], y_shuffled[split_index:]
    
    return x_train, x_test, y_train, y_test

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x_standardized_new, y_standardized_new)

# Plot training data and test data
plt.figure(figsize=(10, 6))
plt.scatter(x_train, y_train, color='blue', label='Training Data')
plt.scatter(x_test, y_test, color='red', label='Test Data')
plt.xlabel('Standardized X')
plt.ylabel('Standardized Y')
plt.title('Training and Test Data')
plt.legend()


Lets repeat our model exploration on just the training data

In [ ]:
def compare_models(degree, alpha_value):
    # Prepare the polynomial features
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly = polynomial_features.fit_transform(x_train[:, np.newaxis])

    # Initialize models with a dynamic alpha value for regularization
    models = {
        'Linear': LinearRegression(),
        'Lasso': Lasso(alpha=alpha_value, max_iter=10000),
        'Ridge': Ridge(alpha=alpha_value)
    }

    # Table to store results
    results = []

    # Fit models and calculate errors and penalties
    for name, model in models.items():
        model.fit(x_poly, y_train)
        y_poly_pred = model.predict(x_poly)

        # Calculating SSE
        sse = np.sum((y_train - y_poly_pred) ** 2)

        # Calculating L1 and L2 penalties
        L1_penalty = np.sum(np.abs(model.coef_))
        L2_penalty = np.sum(np.square(model.coef_))

        # Number of parameters
        num_params = len([param for param in model.coef_ if param != 0])
        
        results.append({
            'Model': name,
            'SSE': sse,
            'Penalty': 'L1' if name == 'Lasso' else 'L2' if name == 'Ridge' else 'None',
            'L1': L1_penalty,
            'L2': L2_penalty,
            'Number of Parameters' : num_params,
        })

    # Create a DataFrame to display the results
    results_df = pd.DataFrame(results)
    display(results_df.set_index('Model'))  # Use display to show the DataFrame in the interactive output

# Create interactive widgets for the degree of the polynomial and alpha value
degree_slider = IntSlider(min=1, max=11, step=1, value=3, description="Polynomial Degree",
                          style={'description_width': 'initial'},
                          layout={'width': '500px'})
lambda_slider = FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description="Lambda",
                           style={'description_width': 'initial'},
                           layout={'width': '500px'})

interactive_plot = interactive(compare_models, degree=degree_slider, alpha_value=lambda_slider)
interactive_plot

Now lets evaluate how this model fits the test data:

In [ ]:
# SET HYPERPARAMETERS HERE:
initial_model_type = 'Standard MLR' # Options are 'Standard MLR', 'Lasso', 'Ridge'
initial_degree = 1
initial_lambda = 0.1
# ------------------------------------

# Standardize the new data
scaler_x = StandardScaler()
x_standardized_new = scaler_x.fit_transform(x_test[:, np.newaxis]).flatten()
scaler_y = StandardScaler()
y_standardized_new = scaler_y.fit_transform(y_test[:, np.newaxis]).flatten()

def plot_model_predictions(model_type, degree, alpha):
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    x_poly_test = polynomial_features.fit_transform(x_test[:, np.newaxis])
    
    if model_type == 'Standard MLR':
        model = LinearRegression()
    elif model_type == 'Lasso':
        model = Lasso(alpha=alpha, max_iter=10000)
    elif model_type == 'Ridge':
        model = Ridge(alpha=alpha)
        
    model.fit(x_poly_test, y_test)
    y_poly_pred_new = model.predict(x_poly_test)

    plt.figure(figsize=(10, 6))
    plt.scatter(x_test, y_test, color='blue', label='Actual Data')
    plt.plot(sorted(x_test), sorted(y_poly_pred_new), color='red', label='Model Prediction')
    plt.title(f'Model Type: {model_type}, Degree: {degree}, Lambda: {alpha}\nR^2: {r2_score(y_standardized_new, y_poly_pred_new):.3f}')
    plt.xlabel('Standardized X')
    plt.ylabel('Standardized Y')
    plt.legend()
    plt.show()

# Interactive widgets setup
model_dropdown = Dropdown(options=['Standard MLR', 'Lasso', 'Ridge'], value=initial_model_type, description='Model Type:')
degree_slider = IntSlider(min=1, max=11, value=initial_degree, description='Polynomial Degree', continuous_update=False)
lambda_slider = FloatSlider(min=0.01, max=1.0, value=initial_lambda, step=0.01, description='Lambda', continuous_update=False)

interactive_plot = interactive(plot_model_predictions, model_type=model_dropdown, degree=degree_slider, alpha=lambda_slider)
display(interactive_plot)


ASSIGNMENT QUESTION:
- Please complete questions 16, 17 and 18 in the assignment.